# Scheduling and Optimisation: Tutor Allocation Problem - Code
* Silvestre Benavente Cartajena
* Isabel Hannebery
* Naryman Tarvand

<i> MAST90050 Scheduling and Optimisation </i>




## Import Libraries

In [1]:
import os
import sys
import copy
import time
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) 2023 Semester 2 Schedule

## Initialization
#### Opens data and defines basic parameters

In [2]:
# Opens data
file              = "Data_Scheduling_AN.xlsx"
rootpath          = "./"
sheets            = pd.ExcelFile(os.path.join(rootpath, file))
availability      = pd.read_excel(sheets,'Availability AN',header = 2)
tutor_class       = pd.read_excel(sheets,'Tutor_Class AN')
rooms             = pd.read_excel(sheets,'Rooms AN')
slots             = pd.read_excel(sheets,'Slots AN')
days              = pd.read_excel(sheets,'Days AN')

# Defines quantity of each variable
num_tutors        = len(tutor_class["Tutor ID"].unique())
num_slots         = len(slots)
num_rooms         = len(rooms)
num_days          = len(days)

# Defines what variable corresponds to what index
ind_tutors        = 0
ind_slots         = 1
ind_days          = 2
ind_rooms         = 3

# Creates decission variable
x = np.zeros((num_tutors, num_slots, num_days, num_rooms))

# Rearranges availability have same index format as decission variable
mon_availability   = np.array(availability[['S01-Mon','S02-Mon','S03-Mon','S04-Mon','S05-Mon']])
tue_availability   = np.array(availability[['S01-Tue','S02-Tue','S03-Tue','S04-Tue','S05-Tue']])
wed_availability   = np.array(availability[['S01-Wed','S02-Wed','S03-Wed','S04-Wed','S05-Wed']])
tutor_availability = np.array([mon_availability,tue_availability,wed_availability])
tutor_availability = np.moveaxis(tutor_availability,0,-1)

print(f'Shape of decission variable x: {np.shape(x)}')
print(f'Shape of tutor availability x: {np.shape(tutor_availability)}')

# Number of times a tutor needs to be scheduled
allocation_num  = np.array(tutor_class['Tutor ID'].value_counts()[availability['Slot']])

# Sets seed
np.random.seed(1000000)

Shape of decission variable x: (21, 5, 3, 6)
Shape of tutor availability x: (21, 5, 3)


## Generation of feasible schedules
#### Randomizes allocation

In [3]:
def generate_random_index(allocation_number,past_indeces):
    # Generates an index that hasn't already been considered
    boolean = False
    
    while not(boolean):
        
        # Generates random index
        random_index = np.random.randint(0, allocation_number)
        
        # If index has not been considered, that index is returned
        # Otherwise repeat process until a feasible index is found
        if random_index not in past_indeces:
            boolean = True
   
    return random_index

In [4]:
def generate_random_room(rooms):
    # Gets an index of a room that is not occupied
    boolean = False
    
    # If there are no rooms available, returns -1
    if sum(rooms) == num_rooms:
        return 'No Rooms Available'
    
    # Looks up rooms which are available
    available_rooms = np.where(rooms == 0)
    
    while not(boolean):
        
        # Generates random index
        random_room = np.random.randint(0, num_rooms)
        
        # If index corresponds to available room, index is returned
        # Otherwise repeat process until a feasible index is found
        if random_room in available_rooms[0]:
            boolean = True
        
    return random_room

In [5]:
def generate_feasible_schedule():
    
    feasible = False
    attempts = 1
    
    while not(feasible):
        
        problem = False
        
        x = np.zeros((num_tutors, num_slots, num_days, num_rooms))
        
        for tutor in range(num_tutors):
            #print(f'Tutor: {tutor}')

            # Gets all slots and days tutor is available
            possible_allocations = np.where(tutor_availability[tutor] == 1)

            # For each tutor, saves indeces of allocated spaces such that tutor is not scheduled to the same time
            past_indeces = []

            #print(f'Possible Allocations: {len(possible_allocations[0])}')
            #print(f'Number of allocations needed: {allocation_num[tutor]}')

            # If allocation number greater than possible allocations: problem is infeasable
            if len(possible_allocations[0]) < allocation_num[tutor]:
                print('Problem is infeasable as number of allocations needed is greater than preference number')
                print('Please review data')
                break

            # Allocates tutor to a room
            for i in range(allocation_num[tutor]):

                # Random index (doesn't double schedule the tutor)
                random_index = generate_random_index(len(possible_allocations[0]),past_indeces)

                # Gets the slot and day of randomized index
                random_slot = possible_allocations[0][random_index]
                random_day  = possible_allocations[1][random_index]

                # Randomizes room for allocation, and selects a room that is available 
                available_rooms = np.apply_over_axes(np.sum, x, ind_tutors)
                available_rooms = available_rooms[0,random_slot,random_day]

                # Gets room that is empty
                random_room = generate_random_room(available_rooms)

                # If can't allocate room: break and restart the porcess
                if random_room == 'No Rooms Available':
                    problem = True
                    #print('No Rooms Available')
                    break

                # Fix so it escapes loop and add while
                x[tutor,random_slot,random_day,random_room] = 1

                past_indeces.append(random_index)
                #print("")
        
        if problem:
            problem  = False
            attempts += 1
        else:
            feasible = True
                
            
    return (attempts,x)


## Printing a feasible schedule
#### Given the decission variable $x_{tsdr}$, it produces a schedule

In [6]:
def print_schedule(instance):
    # Prints out schedule

    # Creates empty schedule
    columns  = pd.MultiIndex(levels = [days['Day'].values,slots['Slot ID'].values],codes = [[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2],[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4]])
    indeces  = rooms['Room ID'].values
    schedule = pd.DataFrame(np.zeros((num_rooms,num_days*num_slots)),columns = columns,index = indeces)
    schedule = schedule.astype(str)

    # Populates empty schedule with tutors dictated by the decission variable
    for day in range(num_days):
        for room in range(num_rooms):
            for slot in range(num_slots):

                # Finds who is the tutor in that room at that slot at that day 
                tutor_index = np.where(instance[:,slot,day,room] == 1)

                # Converts from index form
                day_name  = days['Day'][day]
                slot_name = slots['Slot ID'][slot]
                room_name = rooms['Room ID'][room]

                # Populates cell with tutor name, and if no tutor allocated, leaves it empty    
                if tutor_index[0].size != 0:
                    # Gets tutor Id
                    tutor_id = availability['Slot'][tutor_index[0]]

                    # Updates schdule
                    schedule.loc[room_name,day_name][slot_name] = tutor_id.values[0]

                else:
                    schedule.loc[room_name,day_name][slot_name] = ''  
    return schedule

## Genrating a random feasible schedule

In [7]:
attempts, x = generate_feasible_schedule()
print(f'Number of attempts needed to create feasible schedule: {attempts}')
schedule = print_schedule(x)
schedule

Number of attempts needed to create feasible schedule: 1


Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01                             T17       T12  T18  T04                       
R02         T17                 T10                                           
R03                             T20  T04       T05            T13       T11   
R04         T15                 T04  T07  T05  T02                            
R05    T15                      T16  T21  T21  T06  T02                       
R06    T09                           T16  T06  T13  T08            T01        

               
     S04  S05  
R01  T12       
R02  T19  T19  
R03  T10       
R04  T03       
R05            
R06  T14

# Objective function

## 1) The number of days a tutor has to come into work should be minimized

In [8]:
def total_days_visted_by_tutors(instance):
    
    # Sums over rooms and slots
    sum_over_rooms_and_slots = np.squeeze(np.apply_over_axes(np.sum, instance, [ind_slots,ind_rooms]))

    count = 0
    
    # Iterates over each tutor
    for j in range(num_tutors):
        
        # Counts the days a tutor has to teach
        count += np.count_nonzero(sum_over_rooms_and_slots[j])
    
    return count

## 2) The number of rooms used each day should be minimized

In [9]:
def total_rooms_used_in_week(instance):
    
    # Sums over slots and tutors
    sum_over_tutors_and_slots = np.squeeze(np.apply_over_axes(np.sum, instance, [ind_tutors,ind_slots]))

    count = 0

    # Iterates over each day
    for j in range(num_days):
        
        # Counts the rooms that are used each day
        count += np.count_nonzero(sum_over_tutors_and_slots[j])
    
    return(count)

## 3) The amount of time a tutor waits in between tutorials should me minimized

In [10]:
def tutors_in_idle(instance):
    
    # Sums over every room
    sum_over_rooms = np.squeeze(np.apply_over_axes(np.sum, instance, ind_rooms))

    idle_time = 0
    
    # Iterates over each tutor and each day
    for t in range(num_tutors):
        for d in range(num_days):

            # Gets each tutor's schedule for a given day
            day_schedule = sum_over_rooms[t,:,d].astype(int)

            # Looks at the slots which have been assign to
            indeces = np.where(day_schedule == 1)[0]

            # Calculates the time tutor is waiting around in between tutorials
            if len(indeces) > 1:
                
                idle_time += max(indeces) - min(indeces) + 1 - len(indeces)
                
    return idle_time

## 4) The amount of times tutors should move rooms in between tutorials should be minimized

In [11]:
def times_tutors_move_rooms_with_b2b_tutorials(instance):
    
    times_moved = 0

    # Iterates over each tutor over each day
    for t in range(num_tutors):
            for d in range(num_days):


                # Iterates over each slot
                for s in range(num_slots-1):


                    # If that tutor is allocated to that slot in that day
                    if sum(instance[t,s,d,:] == 1):

                        # Gets what room it is allocated to
                        room_index = np.where(instance[t,s,d,:] == 1)[0]

                        # Check if tutor is allocated on the next slot
                        if sum(instance[t,s+1,d,:] == 1):

                            # Gets what room it is allocated to next slot
                            next_room_index = np.where(instance[t,s+1,d,:] == 1)[0]

                            # Penalizes if rooms are not the same
                            if room_index != next_room_index:
                                times_moved += 1

    return times_moved

### Combining all objectives

In [12]:
def calculate_fitness(individual):
    
    number_of_visits_cost = total_days_visted_by_tutors(individual)
    rooms_used_cost       = total_rooms_used_in_week(individual)
    tutors_waiting_cost   = tutors_in_idle(individual)
    moving_rooms_cost     = times_tutors_move_rooms_with_b2b_tutorials(individual)
    
    #print(f'Total number of visits = {number_of_visits_cost}')
    #print(f'Total number of rooms used over the week = {rooms_used_cost}')
    #print(f'Total time tutors spend waiting = {tutors_waiting_cost}')
    #print(f'Total amount of times tutors having to moves rooms = {moving_rooms_cost}')
    
    return number_of_visits_cost + rooms_used_cost + tutors_waiting_cost + moving_rooms_cost

In [13]:
schedule = print_schedule(x)
schedule

Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01                             T17       T12  T18  T04                       
R02         T17                 T10                                           
R03                             T20  T04       T05            T13       T11   
R04         T15                 T04  T07  T05  T02                            
R05    T15                      T16  T21  T21  T06  T02                       
R06    T09                           T16  T06  T13  T08            T01        

               
     S04  S05  
R01  T12       
R02  T19  T19  
R03  T10       
R04  T03       
R05            
R06  T14

In [14]:
calculate_fitness(x)

48

## Function to check whether an instance is a feasible solution

In [15]:
def is_feasible_schedule(instance):
    
    feasible = True
    
    # Extends tutor availability by one dimension, to be applied to each room
    A = np.repeat(tutor_availability[:, :,:,np.newaxis], num_rooms, axis=3)
    
    # Checks instance satisfies tutors availability
    for t in range(num_tutors):
        for s in range(num_slots):
            for d in range(num_days):
                for r in range(num_rooms):
                    
                    if instance[t,s,d,r] > A[t,s,d,r]:
                        feasible = False
                        return feasible
                        
                        
    # Checks tutor is allocated the amount of times it needs to be allocated
    allocated_number = np.squeeze(np.apply_over_axes(np.sum, instance, [ind_slots,ind_days,ind_rooms]))
    for t in range(num_tutors):
        if allocated_number[t] != allocation_num[t]:
            feasible = False
            return feasible
            
    # Checks that for each room there is at most room tutpr allocated to
    room_usage = np.squeeze(np.apply_over_axes(np.sum, instance, ind_tutors))
    
    for s in range(num_slots):
        for d in range(num_days):
            for r in range(num_rooms):
                
                if room_usage[s,d,r] > 1:
                    feasible = False
                    return feasible
                
    # Checks that a tutor can be at most one room at a time
    tutor_schedule = np.squeeze(np.apply_over_axes(np.sum, instance, ind_rooms))
    
    for t in range(num_tutors):
        for s in range(num_slots):
            for d in range(num_days):
                
                if tutor_schedule[t,s,d] > 1:
                    feasible = False
                    return feasible
                                    
    return feasible

In [16]:
print(f'Is the schedule a feasible solution: {is_feasible_schedule(x)}')

Is the schedule a feasible solution: True


# Genetic Algorithm

In [17]:
def get_fitness_scores(population):
    
    fitness_scores = np.zeros(POPULATION_NUMBER)

    for i in range(len(population)):
        
        fitness_score = -1*calculate_fitness(population[i])
        
        fitness_scores[i] = fitness_score
        
    return fitness_scores
        

In [18]:
def breed(individual1,individual2):
    
    # Initializes child
    child = np.zeros((num_tutors, num_slots, num_days))

     # Relax condition of where tutors are allocated
    parent1 = np.squeeze(np.apply_over_axes(np.sum, individual1, ind_rooms))
    parent2 = np.squeeze(np.apply_over_axes(np.sum, individual2, ind_rooms))
    
    for t in range(num_tutors):

        # Randomly chooses if to take genes from parent one or two
        num = np.random.randint(2)

        if num == 1:
            child[t] = parent1[t]
        else:
            child[t] = parent2[t]
            
    # Extends dimension of child, and assigns rooms sequentially    
    extended_child = np.zeros((num_tutors, num_slots, num_days, num_rooms))

    room_indeces = np.zeros([num_slots,num_days])

    for t in range(num_tutors):
        for s in range(num_slots):
            for d in range(num_days):

                if child[t,s,d] == 1:
                    
                    r = int(room_indeces[s,d])
                    
                    # More than 6 tutors have been allocated to that 
                    if r > num_rooms - 1:
                        return False
                    
                    # Assigns room to that tutor
                    extended_child[t,s,d,r] = 1

                    # Next time someone is allocated to that slot on that time, has to move to the next room
                    room_indeces[s,d] = r + 1
                    
    

    return extended_child    

In [19]:
def generate_child(parent1,parent2):
    
    feasible_offspring = False
    
    while not(feasible_offspring):
        
        offspring = breed(parent1,parent2)
        
        if type(offspring) == bool:
            continue
            
        
        feasible_offspring = is_feasible_schedule(offspring)
        
    return offspring

In [20]:
def mutate(individual):
    
    instance = copy.deepcopy(individual) 
    
    for d in range(num_days):
        for s in range(num_slots):
            for r in range(num_rooms):

                # See if a tutor is allofated to this specific room
                t = np.squeeze(np.where(instance[:,s,d,r] == 1))

                if s< 4:

                    # See if that tutor has back to back tutorials
                    next_slot_room = np.squeeze(np.where(instance[t,s+1,d,:] == 1))

                    # If rooms don't match
                    if r!= next_slot_room and next_slot_room in np.arange(num_rooms):

                        t_other = np.squeeze(np.where(instance[:,s+1,d,r] == 1))

                        if t_other in np.arange(num_tutors):
                            instance[t_other,s+1,d,next_slot_room] = 1
                            instance[t_other,s+1,d,r] = 0

                        # Allocates tutor to new room such that b2b tutorials are in the same room
                        instance[t,s+1,d,r] = 1
                        instance[t,s+1,d,next_slot_room] = 0
                        
    return instance

In [21]:
def roulette_wheel_selection(fitness_scores):
    
    # Randomly picks two individual, an each individual has a weighted probability
    
    pick_num = 2
    prob_dist = np.exp(fitness_scores)/sum(np.exp(fitness_scores))
    indeces   = np.arange(len(fitness_scores))
    
    picks = np.random.choice(indeces,pick_num,p=prob_dist,replace = False)
    
    return picks

In [22]:
# Silences deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Measures run-time of algorithm
start_time = time.time()

# Initializes population
POPULATION_NUMBER = 200
population = []

for i in range(POPULATION_NUMBER):
    _ , x = generate_feasible_schedule()
    population.append(x)
    
# Iterates over generations
MAX_GENERATIONS    = 100

# Saves score over iterations
iterations = []
scores     = []

for generation in range(MAX_GENERATIONS):
    
    next_generation = np.zeros((POPULATION_NUMBER,num_tutors, num_slots, num_days, num_rooms))
    
    # Calculates the fitness score for each individual in the population
    fitness_scores = get_fitness_scores(population)
    
    #print(max(fitness_scores))
    
    # Gets indeces of two highest scoring individuals
    top_indeces      = np.flip(np.argpartition(fitness_scores, -2)[-2:])
    top_index        = top_indeces[0]
    second_top_index = top_indeces[1]
    
    # By elitism, individuals are selected to go onto the next generation
    next_generation[0] = population[top_index]
    next_generation[1] = population[second_top_index]
    
    for i in range(2,POPULATION_NUMBER-10):
        
        parents_indeces = roulette_wheel_selection(fitness_scores)
        parent1         = population[parents_indeces[0]]
        parent2         = population[parents_indeces[1]]
        
        # Creates a feasible offspring from the two parents
        offspring         = generate_child(parent1,parent2)
        mutated_offspring = mutate(offspring)
        
        # Appends child to next generation
        next_generation[i] = mutated_offspring
        
    for i in range(POPULATION_NUMBER-10,POPULATION_NUMBER):
        _ , x = generate_feasible_schedule()
        
        # Appends child to next generation
        next_generation[i] = x
        
    population = next_generation  
    
    iterations.append(generation)
    scores.append(max(fitness_scores))
    
# End time of algorithm
end_time = time.time()

## Best Schedule

In [23]:
schedule   = print_schedule(population[top_index])
best_score = calculate_fitness(population[top_index])
schedule

Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01    T09                      T05  T05  T06  T06  T20       T13  T13  T11   
R02    T17  T17                 T14  T07  T12  T12  T08            T10  T10   
R03    T15  T15                 T18  T21  T21  T02  T02                       
R04                             T16  T16  T04  T04  T04                       
R05                                                                           
R06                                                                           

               
     S04  S05  
R01  T01  T03  
R02  T19  T19  
R03            
R04            
R05            
R06

In [24]:
print('Total Cost of Best Solution:', best_score)
print('Genetic Algorithm Run-Time:',  round(end_time - start_time,3), 'seconds')

Total Cost of Best Solution: 30
Genetic Algorithm Run-Time: 41.521 seconds


# Tabu Search

# Grasp

# 2) Case Study: Expansion Plan

## Initialization
#### Opens data and defines basic parameters

In [25]:
# Opens data
file              = "Data_Scheduling_AN_New_Instance.xlsx"
rootpath          = "./"
sheets            = pd.ExcelFile(os.path.join(rootpath, file))
availability      = pd.read_excel(sheets,'Availability AN',header = 2)
tutor_class       = pd.read_excel(sheets,'Tutor_Class AN')
rooms             = pd.read_excel(sheets,'Rooms AN')
slots             = pd.read_excel(sheets,'Slots AN')
days              = pd.read_excel(sheets,'Days AN')

# Defines quantity of each variable
num_tutors        = len(tutor_class["Tutor ID"].unique())
num_slots         = len(slots)
num_rooms         = len(rooms)
num_days          = len(days)

# Defines what variable corresponds to what index
ind_tutors        = 0
ind_slots         = 1
ind_days          = 2
ind_rooms         = 3

# Creates decission variable
x = np.zeros((num_tutors, num_slots, num_days, num_rooms))

# Rearranges availability have same index format as decission variable
mon_availability   = np.array(availability[['S01-Mon','S02-Mon','S03-Mon','S04-Mon','S05-Mon']])
tue_availability   = np.array(availability[['S01-Tue','S02-Tue','S03-Tue','S04-Tue','S05-Tue']])
wed_availability   = np.array(availability[['S01-Wed','S02-Wed','S03-Wed','S04-Wed','S05-Wed']])
tutor_availability = np.array([mon_availability,tue_availability,wed_availability])
tutor_availability = np.moveaxis(tutor_availability,0,-1)

print(f'Shape of decission variable x: {np.shape(x)}')
print(f'Shape of tutor availability x: {np.shape(tutor_availability)}')

# Number of times a tutor needs to be scheduled
allocation_num  = np.array(tutor_class['Tutor ID'].value_counts()[availability['Slot']])

Shape of decission variable x: (40, 5, 3, 8)
Shape of tutor availability x: (40, 5, 3)


# Genetic Algorithm

In [26]:
# Sets seed
np.random.seed(1000000)

# Silences deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Measures run-time of algorithm
start_time = time.time()

# Initializes population
POPULATION_NUMBER = 200
population = []

for i in range(POPULATION_NUMBER):
    _ , x = generate_feasible_schedule()
    population.append(x)
    
# Iterates over generations
MAX_GENERATIONS    = 100

# Saves score over iterations
iterations = []
scores     = []

for generation in range(MAX_GENERATIONS):
    
    next_generation = np.zeros((POPULATION_NUMBER,num_tutors, num_slots, num_days, num_rooms))
    
    # Calculates the fitness score for each individual in the population
    fitness_scores = get_fitness_scores(population)
    
    #print(max(fitness_scores))
    
    # Gets indeces of two highest scoring individuals
    top_indeces      = np.flip(np.argpartition(fitness_scores, -2)[-2:])
    top_index        = top_indeces[0]
    second_top_index = top_indeces[1]
    
    # By elitism, individuals are selected to go onto the next generation
    next_generation[0] = population[top_index]
    next_generation[1] = population[second_top_index]
    
    for i in range(2,POPULATION_NUMBER-10):
        
        parents_indeces = roulette_wheel_selection(fitness_scores)
        parent1         = population[parents_indeces[0]]
        parent2         = population[parents_indeces[1]]
        
        # Creates a feasible offspring from the two parents
        offspring         = generate_child(parent1,parent2)
        mutated_offspring = mutate(offspring)
        
        # Appends child to next generation
        next_generation[i] = mutated_offspring
        
    for i in range(POPULATION_NUMBER-10,POPULATION_NUMBER):
        _ , x = generate_feasible_schedule()
        
        # Appends child to next generation
        next_generation[i] = x
        
    population = next_generation  
    
    iterations.append(generation)
    scores.append(max(fitness_scores))
    
# End time of algorithm
end_time = time.time()

In [27]:
schedule   = print_schedule(population[top_index])
best_score = calculate_fitness(population[top_index])
schedule

Monday                  Tuesday                     Wednesday            \
       S01  S02 S03 S04 S05     S01  S02  S03  S04  S05       S01  S02  S03   
R01    T17  T17                 T05  T05  T01  T20  T08       T23  T07  T29   
R02    T15  T15                 T14  T21  T21  T02  T02       T26  T12  T12   
R03    T40  T09                 T10  T10  T06  T06  T27       T28  T31  T31   
R04                             T16  T16  T13  T13  T24       T38  T38  T39   
R05                             T30  T18  T04  T04  T04                       
R06                             T33  T29  T29  T37  T37                       
R07                             T35  T32  T22  T22  T36                       
R08                                                                           

               
     S04  S05  
R01  T19  T19  
R02  T11  T03  
R03  T25  T25  
R04  T34  T34  
R05            
R06            
R07            
R08

In [28]:
print('Total Cost of Best Solution:', best_score)
print('Genetic Algorithm Run-Time:',  round(end_time - start_time,3), 'seconds')

Total Cost of Best Solution: 55
Genetic Algorithm Run-Time: 78.812 seconds
